# Connect to RDS Postgres Database Server

In [14]:
import boto3
import json
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [2]:
def get_secret(secret_name, region_name="us-east-1"):
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name)
    get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    get_secret_value_response = json.loads(get_secret_value_response['SecretString'])
    return get_secret_value_response

In [12]:
db_credentials = get_secret(secret_name='wysde')

USERNAME = db_credentials["RDS_POSTGRES_USERNAME"]
PASSWORD = db_credentials["RDS_POSTGRES_PASSWORD"]
HOST = db_credentials["RDS_POSTGRES_HOST"]
PORT = 5432
DBNAME = "dec22"

CONN = f"postgresql://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}"

In [15]:
# load the data into our postgres database
alchemyEngine = create_engine(CONN, pool_recycle=3600);
postgreSQLConnection = alchemyEngine.connect();

---

In [ ]:
df = pd.read_csv("s3://wysde-datasets/acled/2022-09-01-2022-10-26.csv", sep=";")
df

In [ ]:
df.info()

In [16]:
df.head(10).to_sql('acled', postgreSQLConnection, if_exists='replace', index=False)

---

In [17]:
df = pd.read_parquet("s3://wysde2-test/datalake/sparsh/refined/services/data.parquet.snappy")
df

,service_id,service_name
0,0,speech therapy
1,1,dietetics
2,2,physiotherapy
3,3,occupational therapy
4,4,music therapy
